In [1]:
import pandas as pd
import numpy as np
import nltk
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords, wordnet
import string
from nltk import pos_tag
from nltk.stem import WordNetLemmatizer
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
nltk.download('punkt')
nltk.download('stopwords')
nltk.download('averaged_perceptron_tagger')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!


True

In [2]:
data = pd.read_csv('twitter_x_y_train.csv')

In [3]:
y = list(data['airline_sentiment'])

In [4]:
def get_simple_pos(tag): #creating simple tags to pass into the lemmatizer
    if tag.startswith('J'):
        return wordnet.ADJ
    elif tag.startswith('V'):
        return wordnet.VERB
    elif tag.startswith('N'):
        return wordnet.NOUN
    elif tag.startswith('R'):
        return wordnet.ADV
    else:
        return wordnet.NOUN

In [5]:
lemmatizer = WordNetLemmatizer()
def clean_tweet(tweet):
    words = word_tokenize(tweet)
    output_words = []
    for w in words:
        if w.lower() not in list(string.punctuation):
            pos = pos_tag([w])
            clean_word = lemmatizer.lemmatize(w, get_simple_pos(pos[0][1]))
            output_words.append(clean_word.lower())
    return " ".join(output_words)

In [6]:
x_data = []
for twt in data['text']:
    x_data.append(clean_tweet(twt))

In [7]:
#x_train, x_test, y_train, y_test = train_test_split(x_data, y, random_state= 1)

In [8]:
vectorizer = TfidfVectorizer(max_features= 5000, ngram_range=(1, 2), max_df = 0.7)

In [9]:
x_train_vect = vectorizer.fit_transform(x_data)
#x_test_vect = vectorizer.transform(x_test)

In [10]:
x_test = pd.read_csv('/content/twitter_x_test.csv')
x_test_data = []
for twt in x_test['text']:
    x_test_data.append(clean_tweet(twt))
x_test_vect = vectorizer.transform(x_test_data)

In [11]:
from sklearn import svm
svc = svm.SVC()
svc.fit(x_train_vect, y)
#svc.score(x_test_vect, y_test)
y_pred = svc.predict(x_test_vect)

In [12]:
pd_series = pd.Series(y_pred)
pd_series.to_csv('predicted_sentiments.csv', index = False, header= False)